In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime
import urllib.request

now = datetime.datetime.now().strftime("%Y%m%d")

In [19]:
#new chrome driver
def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    option = webdriver.ChromeOptions()

    #fix chrome 103 problem
    option.binary_location = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

    prefs={"profile.default_content_setting_values.notifications" : 2}
    option.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=option)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [20]:
#return name and download icon

def getPhoneDetailData(brand_name, soup):
    #param
    phone_list =[]

    phones = soup.find_all("div", class_="flex_gadget_wrap")
    if(phones):
        for phone in phones:
            phone_img_div = phone.find("div", class_="image")
            phone_img = phone_img_div.find("img")["src"]

            phone_data_div = phone.find("div", class_="model_no")
            phone_data_a = phone_data_div.find("a")
            phone_name = phone_data_a.get_text()
            phone_link = "https://www.dcfever.com"+phone_data_a["href"]
            img_save_name = "../icon/"+phone_name+".jpg"
            time.sleep(0.5)
            urllib.request.urlretrieve(phone_img, img_save_name)

            phone_list.append([brand_name, phone_name, phone_link, img_save_name])
            
    return phone_list

In [21]:
#return phone of a brand

def getBrandPhoneData(brand_name, url, driver):
    phone_list =[]
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    pages_div = soup.find("div", class_="pages")
    pages = pages_div.find_all("a")[:-1]

    if pages:
        for page in pages:
            temp_link = "https://www.dcfever.com/"+page["href"]
            driver.get(temp_link)
            time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            phone_list+=getPhoneDetailData(brand_name, soup)
    
    else:
        phone_list+=getPhoneDetailData(brand_name, soup)

    return phone_list



In [22]:
#return brands

def getBrandData(url, driver):

    #param
    brand_list = []
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    brand_matrix = soup.find("ul", class_="brand_matrix clearfix")
    
    for brand in brand_matrix.find_all("li"):

        #brand name
        temp_brand_name_link = brand.find("img")["src"]
        temp_brand_name = temp_brand_name_link.replace("https://cdn09.dcfever.com/media/brands/","")
        temp_brand_name = temp_brand_name.replace(".gif","")
        temp_brand_name = temp_brand_name.replace(".jpg","")

        #brand link
        temp_brand_link = "https://www.dcfever.com/phones/"+brand.find("a")["href"]
        brand_list.append([temp_brand_name, temp_brand_link])

    #save brand
    brand_df = pd.DataFrame(data=brand_list, columns=['Brand_Name', 'Link'])
    file_name = "../crawler_data/Phone_Brand_Data"+now+".csv"
    brand_df.to_csv(file_name, index=False, encoding="utf-8")

    return True

if(getBrandData("https://www.dcfever.com/phones/database.php", OpenChromeDriver())):
    print("done")
else:
    print("fail")

In [23]:
brand_df = pd.read_csv("../crawler_data/Phone_Brand_Data20220719.csv")
brand_df

,Brand_Name,Link
0,apple,https://www.dcfever.com/phones/brand.php?id=49
1,asus,https://www.dcfever.com/phones/brand.php?id=50
2,google,https://www.dcfever.com/phones/brand.php?id=173
3,honorlogo,https://www.dcfever.com/phones/brand.php?id=312
4,htc,https://www.dcfever.com/phones/brand.php?id=61
5,huawei_logo,https://www.dcfever.com/phones/brand.php?id=151
6,microsoft,https://www.dcfever.com/phones/brand.php?id=141
7,motorola,https://www.dcfever.com/phones/brand.php?id=66
8,nokia,https://www.dcfever.com/phones/brand.php?id=68
9,Nothing_Logo.webp.png,https://www.dcfever.com/phones/brand.php?id=328


In [24]:
#param
phone_list = []

for i in brand_df.index:
    try:
        phone_list+= getBrandPhoneData(brand_df["Brand_Name"][i], brand_df["Link"][i], OpenChromeDriver())
        time.sleep(1)
    except:
        print(brand_df["Brand_Name"][i])
        
#save phone
phone_df = pd.DataFrame(data=phone_list, columns=['Brand_Name', 'Phone_Name', 'Link', 'Icon'])
file_name = "../crawler_data/Phone_Data"+now+".csv"
brand_df.to_csv(file_name, index=False, encoding="utf-8")

print("done")

apple
asus
google
honorlogo
htc


In [ ]:
#save phone
phone_df = pd.DataFrame(data=phone_list, columns=['Brand_Name', 'Phone_Name', 'Link', 'Icon'])
file_name = "../crawler_data/Phone_Data"+now+".csv"
phone_df.to_csv(file_name, index=False, encoding="utf-8")

In [3]:
phone_df = pd.read_csv("../crawler_data/Phone_Data20220719.csv")
phone_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,Apple iPhone 3G,https://www.dcfever.com/phones/specification.p...,../icon/Apple iPhone 3G.jpg
1,apple,Apple iPhone 2G,https://www.dcfever.com/phones/specification.p...,../icon/Apple iPhone 2G.jpg
2,google,Google Pixel 6 Pro,https://www.dcfever.com/phones/specification.p...,../icon/Google Pixel 6 Pro.jpg
3,google,Google Pixel 6,https://www.dcfever.com/phones/specification.p...,../icon/Google Pixel 6.jpg
4,google,Google Pixel 4a 5G,https://www.dcfever.com/phones/specification.p...,../icon/Google Pixel 4a 5G.jpg
...,...,...,...,...
523,miui,小米 3,https://www.dcfever.com/phones/specification.p...,../icon/小米 3.jpg
524,miui,MIUI 小米 2,https://www.dcfever.com/phones/specification.p...,../icon/MIUI 小米 2.jpg
525,miui,MIUI 小米 2S,https://www.dcfever.com/phones/specification.p...,../icon/MIUI 小米 2S.jpg
526,miui,紅米 Note 5,https://www.dcfever.com/phones/specification.p...,../icon/紅米 Note 5.jpg
